In [ ]:
# Скрипт для парсинга одного известного рейтингого агенства - 
# происходил сбор данных о пресс-релизах и присваемаеых в них рейтингах

In [ ]:
from time import sleep
import requests
from warnings import warningfilter
warningfilter('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support  import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [ ]:
url_of_page = ''

browser = Chrome('chromedriver.exe')
browser.get(url_of_page)
page = browser.page_source
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
output = pd.DataFrame()
number_of_written_comps = 0
for row in tqdm(list(soup.find('div', {'class':'search-table__wrapper-main', 'data-type':'emits'}).children)):
    company_info {'name':[], 'tax_code':[], 'sector':[], 'rating_final':[],
                  'press_release_final':[], 'scale':[], 'rating':[], 'press_release':[]}
    
    company_url = 'https://...' + row.find('a').attrs['href']
    ratePerson = row.find('a', {'class':'emits-row__item'}).text.strip()
    rating_final = row.p.text
    sector = row.find('div', {'class':'emits-row__item', 'data-type':'sector'}).text.strip()
    press_realese = row.find('div', {'class':'emits-row__item', 'data-type':'pressRelease'}).text.strip()
    
    company_html = requests.get(company_url, verify=False)
    company_soup = BeautifulSoup(company_html.content, 'html.parser')
    if 'ИНН' not in ''.join([el.text for el in list(company_soup.find_all('div', {'class':'info info-bottom'}))]):
        continue
    try:
        for el in list(company_soup.find_all('div', {'class':'info info-bottom'})):
            if 'ИНН' in el.text:
                tax_code = el.p.text.strip()
    except: continue
    try:
        for el in [el for el in list(company_soup.find('div', {'class':'rating-list rating-list--js'}).children) if el!='\n']
        
            scale = el.find('div', {'class':'item-info', 'data-type':'type'}).text.strip()
            rating = el.find('div', {'class':'rating-widget'}).text.strip()
            date = el.find('a', {'data-type':'pressRelease'}).text.strip()
            
            company_info['name'].append(ratePerson)
            company_info['tax_code'].append(tax_code)
            company_info['sector'].append(sector)
            company_info['rating_final'].append(rating_final)
            company_info['press_release_final'].append(press_release)
            company_info['scale'].append(scale)
            company_info['rating'].append(rating)
            company_info['press_release'].append(date)
    except:
        company_info['name'].append(ratePerson)
        company_info['tax_code'].append(tax_code)
        company_info['sector'].append(sector)
        company_info['rating_final'].append(rating_final)
        company_info['press_release_final'].append(press_release)
        company_info['scale'].append('Национальная рейтинговая шкала')
        company_info['rating'].append(rating)
        company_info['press_release'].append(date)
    
    output = pd.concat([output, pd.DataFrame(comapny_info)])
    rnd = np.random.poisson(2.0)
    number_of_written_comps += 1
    sleep(rnd)
print('Записано %i команий.' % number_of_written_comps)
output['new_rating'] = output['rating'].apply(lambda x: x[:x.find(' ')])
output['status'] = output['rating'].apply(lambda x: x[x.find(' '):].strip())
output.rename({'rating':'rating_old', 'new_rating':'rating'}, axis=1, inplace=True)
output.to_csv('rating_table.csv')